Initial notebook for the Unitary Fund's QRISE 2024 challenge 


Unitary Fund mitiq GitHub
https://github.com/unitaryfund/mitiq?tab=readme-ov-file

Qiskit Documentation
https://docs.quantum.ibm.com/

Cirq Documentation
https://quantumai.google/cirq/start/basics

mitiq Documentation
https://github.com/natestemen/talks/blob/main/2024-03-13-qrise/demo.ipynb



TODO: Get running



Basic ZNE using Cirq


In [10]:
import mitiq
from mitiq.zne.inference import RichardsonFactory, LinearFactory, ExpFactory
import cirq
frontend="cirq"

def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""

    # add depolarizing noise
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))

    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

def execute_no_noise(circuit):
    return (
        cirq.DensityMatrixSimulator()
        .simulate(circuit)
        .final_density_matrix[0, 0]
        .real
    )

a, b, c = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    cirq.H(a),
    cirq.CNOT(a, b),
    cirq.CNOT(b, c),
    cirq.S(a),
)

factory = RichardsonFactory([1, 4, 7])
factory.run(circuit, execute_no_noise)
control = factory.reduce()

factory = RichardsonFactory([1, 4, 7])
factory.run(circuit, execute)
first = factory.reduce()

factory = LinearFactory([1, 4, 7])
factory.run(circuit, execute)
second = factory.reduce()

factory = ExpFactory([1, 4, 7])
factory.run(circuit, execute)
third = factory.reduce()

triple_zne = (first + second + third) / 3

print(f"control: {control}")
print(f"first: {first}")
print(f"second: {second}")
print(f"third: {third}")
print(f"avg: {triple_zne}")
print(f"error 1 = {abs(control - first)} error 2 = {abs(control - second)} error 3 = {abs(control - third)} error tzne = {abs(control - triple_zne)}")

c:\Users\readi\AppData\Local\Programs\Python\Python310\lib\site-packages\mitiq\zne\inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
c:\Users\readi\AppData\Local\Programs\Python\Python310\lib\site-packages\mitiq\zne\inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
c:\Users\readi\AppData\Local\Programs\Python\Python310\lib\site-packages\mitiq\zne\inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
c:\Users\readi\AppData\Local\Programs\Python\Python310\lib\site-packages\mitiq\zne\inference.py:96: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


control: 0.4999999867545234
first: 0.5019015371799469
second: 0.49546728531519574
third: 0.5041378142952675
avg: 0.50050221226347
error 1 = 0.0019015504254235038 error 2 = 0.004532701439327658 error 3 = 0.004137827540744099 error tzne = 0.0005022255089466299


c:\Users\readi\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minpack_py.py:906: ExtrapolationWarning: The extrapolation fit may be ill-conditioned. Likely, more data points are necessary to fit the parameters of the model.
  warnings.warn('Covariance of the parameters could not be estimated',


In [ ]:
factory.plot_fit()

What if we tried the above with a larger circuit i.e. doubling the circuit above up?

In [16]:
import mitiq
from mitiq.zne.inference import RichardsonFactory, LinearFactory, ExpFactory
import cirq
frontend="cirq"

def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""

    # add depolarizing noise
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))

    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

def execute_no_noise(circuit):
    return (
        cirq.DensityMatrixSimulator()
        .simulate(circuit)
        .final_density_matrix[0, 0]
        .real
    )

a, b, c = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    cirq.H(a),
    cirq.CNOT(a, b),
    cirq.CNOT(b, c),
    cirq.S(a),
    cirq.H(a),
    cirq.CNOT(a, b),
    cirq.CNOT(b, c),
    cirq.S(a),
)

factory = RichardsonFactory([1, 4, 7])
factory.run(circuit, execute_no_noise)
control = factory.reduce()

factory = RichardsonFactory([1, 4, 7])
factory.run(circuit, execute)
first = factory.reduce()

factory = LinearFactory([1, 4, 7])
factory.run(circuit, execute)
second = factory.reduce()

factory = ExpFactory([1, 4, 7])
factory.run(circuit, execute)
third = factory.reduce()

triple_zne = (first + second + third) / 3

print(f"control: {control}")
print(f"first: {first}")
print(f"second: {second}")
print(f"third: {third}")
print(f"avg: {triple_zne}")
print(f"error 1 = {abs(control - first)} error 2 = {abs(control - second)} error 3 = {abs(control - third)} error tzne = {abs(control - triple_zne)}")

control: 0.24999997019767756
first: 0.25063232084115356
second: 0.24677690366903943
third: 0.25050542210516247
avg: 0.24930488220511848
error 1 = 0.0006323506434759985 error 2 = 0.00322306652863813 error 3 = 0.0005054519074849106 error tzne = 0.0006950879925590736


c:\Users\readi\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_minpack_py.py:906: ExtrapolationWarning: The extrapolation fit may be ill-conditioned. Likely, more data points are necessary to fit the parameters of the model.
  warnings.warn('Covariance of the parameters could not be estimated',


How do richardson, exp, and linear compare to just linear?

In [17]:
import mitiq
from mitiq.zne.inference import RichardsonFactory, LinearFactory, ExpFactory
import cirq
frontend="cirq"

def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""

    # add depolarizing noise
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))

    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )

def execute_no_noise(circuit):
    return (
        cirq.DensityMatrixSimulator()
        .simulate(circuit)
        .final_density_matrix[0, 0]
        .real
    )

a, b, c = cirq.LineQubit.range(3)

circuit = cirq.Circuit(
    cirq.H(a),
    cirq.CNOT(a, b),
    cirq.CNOT(b, c),
    cirq.S(a),
    cirq.H(a),
    cirq.CNOT(a, b),
    cirq.CNOT(b, c),
    cirq.S(a),
)

factory = LinearFactory([1, 4, 7])
factory.run(circuit, execute_no_noise)
control = factory.reduce()

factory = LinearFactory([1, 4, 7])
factory.run(circuit, execute)
first = factory.reduce()

factory = LinearFactory([1, 4, 7])
factory.run(circuit, execute)
second = factory.reduce()

factory = LinearFactory([1, 4, 7])
factory.run(circuit, execute)
third = factory.reduce()

triple_zne = (first + second + third) / 3

print(f"control: {control}")
print(f"first: {first}")
print(f"second: {second}")
print(f"third: {third}")
print(f"avg: {triple_zne}")
print(f"error 1 = {abs(control - first)} error 2 = {abs(control - second)} error 3 = {abs(control - third)} error tzne = {abs(control - triple_zne)}")

control: 0.24999997019767756
first: 0.24698526163895917
second: 0.24595733483632398
third: 0.2463643799225489
avg: 0.24643565879927734
error 1 = 0.003014708558718382 error 2 = 0.004042635361353575 error 3 = 0.003635590275128664 error tzne = 0.003564311398400216


DDD with mitiq


In [6]:
from mitiq.benchmarks import generate_ghz_circuit
from mitiq import ddd


num_qubits = 6
ghz = generate_ghz_circuit(num_qubits)

#print(ghz)
ddd.insert_ddd_sequences(ghz, rule=ddd.rules.xx)
ddd.execute_with_ddd(ghz, execute, rule=ddd.rules.xx)

0.4451800286769867

What about DDD plus ZNE?  What if we triple this up?

After testing, it doesn't seem to impart much improvement.

In [23]:
from mitiq import zne
from mitiq.zne.scaling import fold_global, fold_gates_at_random, insert_id_layers
from mitiq import ddd


@ddd.ddd_decorator(rule=ddd.rules.xyxy)
@zne.zne_decorator(factory=RichardsonFactory([1, 3, 7]), scale_noise=insert_id_layers)
def new_execute(circuit):
    return execute(circuit)

new_execute(circuit)

0.2498570270836352

In [18]:
import cirq
from mitiq import zne
from mitiq import ddd
from mitiq.zne.inference import RichardsonFactory
from mitiq.zne.scaling import fold_global, fold_gates_at_random, insert_id_layers


a, b, c = cirq.LineQubit.range(3)
circuit = cirq.Circuit(
    cirq.H(a),
    cirq.CNOT(a, b),
    cirq.CNOT(b, c),
    cirq.S(a),
    cirq.H(a),
    cirq.CNOT(a, b),
    cirq.CNOT(b, c),
    cirq.S(a),
)

control = execute_no_noise(circuit)
print(f"control: {control}")

@ddd.ddd_decorator(rule=ddd.rules.xyxy)
@zne.zne_decorator(factory=RichardsonFactory([1, 3, 7]), scale_noise=insert_id_layers)
def new_execute(circuit):
    first = execute(circuit)
    second = execute(circuit)
    third = execute(circuit)

    triple_zne = (first + second + third) / 3

    print(f"first: {first} second: {second} third: {third}")
    print(f"f_error: {abs(first - control)} s_error: {abs(second - control)} t_error: {abs(third - control)}")
    print(f"triple_zne: {triple_zne} t_zne error: {abs(triple_zne - control)}")

    return (first + second + third) / 3

new_execute(circuit)

control: 0.2499999701976776
first: 0.24113260209560394 second: 0.24113260209560394 third: 0.24113260209560394
f_error: 0.00886736810207367 s_error: 0.00886736810207367 t_error: 0.00886736810207367
triple_zne: 0.2411325971285502 t_zne error: 0.00886737306912741
first: 0.22524039447307587 second: 0.22524039447307587 third: 0.22524039447307587
f_error: 0.024759575724601746 s_error: 0.024759575724601746 t_error: 0.024759575724601746
triple_zne: 0.22524038950602213 t_zne error: 0.024759580691655486
first: 0.1996825486421585 second: 0.1996825486421585 third: 0.1996825486421585
f_error: 0.050317421555519104 s_error: 0.050317421555519104 t_error: 0.050317421555519104
triple_zne: 0.19968255360921225 t_zne error: 0.050317416588465363


0.24985702335834478